In [0]:
print('فاصل اعلاني')

فاصل اعلاني


In [0]:
spark.conf.set(
  "Your Account Key",
  "your_Access Key"
)

In [0]:
df_store = spark.read.parquet("abfss://retail@elmougist.dfs.core.windows.net/dbo.stores.parquet")
df_store.show(5)

+--------+--------------------+------------+
|store_id|          store_name|    location|
+--------+--------------------+------------+
|       1|     City Mall Store|         UAE|
|       2|   High Street Store|Saudi Arabia|
|       3|   Tech World Outlet|       Qatar|
|       4|Cairo Festival Ci...|       Egypt|
|       5|          Mega Plaza|      Kuwait|
+--------+--------------------+------------+



In [0]:
df_product = spark.read.parquet("abfss://retail@elmougist.dfs.core.windows.net/dbo.products.parquet")
df_product.show(5)

+----------+-----------------+-----------+-----+
|product_id|     product_name|   category|price|
+----------+-----------------+-----------+-----+
|         1|   Wireless Mouse|Electronics|  800|
|         2|Bluetooth Speaker|Electronics| 1200|
|         3|         Yoga Mat|    Fitness|  499|
|         4|     Laptop Stand|Accessories|  999|
|         5|     Notebook Set| Stationery|  149|
+----------+-----------------+-----------+-----+
only showing top 5 rows


In [0]:
df_transaction = spark.read.parquet("abfss://retail@elmougist.dfs.core.windows.net/dbo.transactions.parquet")
df_transaction.show(5)

+--------------+-----------+----------+--------+--------+----------------+
|transaction_id|customer_id|product_id|store_id|quantity|transaction_date|
+--------------+-----------+----------+--------+--------+----------------+
|            31|        101|         1|       1|       3|      2025-04-01|
|            32|        102|         2|       2|       2|      2025-04-03|
|            33|        103|         3|       3|       1|      2025-04-05|
|            34|        104|         4|       4|       5|      2025-04-07|
|            35|        105|         5|       5|       2|      2025-04-09|
+--------------+-----------+----------+--------+--------+----------------+
only showing top 5 rows


In [0]:
df_store = spark.read.parquet("abfss://retail@elmougist.dfs.core.windows.net/dbo.stores.parquet")
df_store.show(5)

+--------+--------------------+------------+
|store_id|          store_name|    location|
+--------+--------------------+------------+
|       1|     City Mall Store|         UAE|
|       2|   High Street Store|Saudi Arabia|
|       3|   Tech World Outlet|       Qatar|
|       4|Cairo Festival Ci...|       Egypt|
|       5|          Mega Plaza|      Kuwait|
+--------+--------------------+------------+



In [0]:
df_customers = spark.read.parquet("abfss://retail@elmougist.dfs.core.windows.net/MohammedHameds/Retails_Project/refs/heads/main/dataset/customers.parquet")
df_customers.show(5)

+-----------+----------------+--------------------+------------+-----------------+
|customer_id|       full_name|               email|     country|registration_date|
+-----------+----------------+--------------------+------------+-----------------+
|        101|    Ahmed Khaled|ahmed.khaled1@gma...|       Egypt|       2025-10-16|
|        102| Sara Al Mansour|sara.almansour2@o...|Saudi Arabia|       2025-10-18|
|        103|    Layla Kazemi|layla.kazemi3@yah...|         UAE|       2025-10-19|
|        104|     Omar Farouk|omar.farouk4@gmai...|       Egypt|       2025-10-17|
|        105|Fatima Al Rashid|fatima.alrashid5@...|Saudi Arabia|       2025-10-15|
+-----------+----------------+--------------------+------------+-----------------+
only showing top 5 rows


In [0]:
df_store.write.mode("overwrite").saveAsTable("`elmougi-retail`.`elmougi-sales`.`BRONZE`")

In [0]:
df_transaction.write.mode("overwrite").saveAsTable("`elmougi-retail`.`elmougi-sales`.`BRONZE-transaction`")

In [0]:
df_product.write.mode("overwrite").saveAsTable("`elmougi-retail`.`elmougi-sales`.`BRONZE-products`")

In [0]:
df_customers.write.mode("overwrite").saveAsTable("`elmougi-retail`.`elmougi-sales`.`BRONZE-customers`")

In [0]:
df_store.printSchema()

root
 |-- store_id: integer (nullable = true)
 |-- store_name: string (nullable = true)
 |-- location: string (nullable = true)



In [0]:
df_product.printSchema()
df_transaction.printSchema()
df_customers.printSchema()

root
 |-- product_id: integer (nullable = true)
 |-- product_name: string (nullable = true)
 |-- category: string (nullable = true)
 |-- price: integer (nullable = true)

root
 |-- transaction_id: integer (nullable = true)
 |-- customer_id: integer (nullable = true)
 |-- product_id: integer (nullable = true)
 |-- store_id: integer (nullable = true)
 |-- quantity: integer (nullable = true)
 |-- transaction_date: date (nullable = true)

root
 |-- customer_id: long (nullable = true)
 |-- full_name: string (nullable = true)
 |-- email: string (nullable = true)
 |-- country: string (nullable = true)
 |-- registration_date: string (nullable = true)



In [0]:
from pyspark.sql.functions import  col , sum

In [0]:
df_customers= df_customers.select(
    col("customer_id").cast("int"),
    col("full_name"),
    col("email"),
    col("country"),
    col("registration_date").cast("date")
)

df_customers.printSchema()

root
 |-- customer_id: integer (nullable = true)
 |-- full_name: string (nullable = true)
 |-- email: string (nullable = true)
 |-- country: string (nullable = true)
 |-- registration_date: date (nullable = true)



In [0]:
df_retail_silver= df_transaction.join(df_customers,"customer_id").join(df_store,"store_id").join(df_product,"product_id").write.mode("overwrite").saveAsTable("`elmougi-retail`.`elmougi-sales`.`retail_silver`")

In [0]:
df_retail_silver = spark.table(
    "`elmougi-retail`.`elmougi-sales`.`retail_silver`"
)

from pyspark.sql.functions import sum, col

df_store_sales_gold = (
    df_retail_silver.groupBy("store_name")
    .agg(
        sum("price").alias("total_sales")
    )
    .orderBy(
        col("total_sales").desc()
    )
)

display

df_store_sales_gold.write.mode("overwrite").saveAsTable("`elmougi-retail`.`elmougi-sales`.`storeSales_gold`")


In [0]:
df_retail_silver = spark.table(
    "`elmougi-retail`.`elmougi-sales`.`retail_silver`"
)

from pyspark.sql.functions import sum, col, expr

df_country_revenue = (
    df_retail_silver.groupBy("country")
    .agg(
        sum(expr("price * quantity")).alias("total_revenue")
    )
    .orderBy(
        col("total_revenue").desc()
    )
)

# Prepare data for waterfall chart: sort by total_revenue and add cumulative sum
from pyspark.sql.window import Window

window_spec = Window.orderBy(col("total_revenue").desc())
df_waterfall = df_country_revenue.withColumn(
    "cumulative_revenue", sum("total_revenue").over(window_spec)
)

display(df_waterfall)
df_waterfall.write.mode("overwrite").saveAsTable("`elmougi-retail`.`elmougi-sales`.`countryRevenue_waterfall_gold`")

/databricks/spark/python/pyspark/sql/connect/expressions.py:1061: UserWarning: WARN WindowExpression: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
  warnings.warn(


country,total_revenue,cumulative_revenue
Saudi Arabia,74010,74010
Egypt,56414,130424
UAE,47161,177585
